# 1. Setup Amazon Athena

Amazon Athena에서는 interactive query service 제공합니다. 표준 SQL을 사용하여 Amazon S3에서 데이터를 분석하는 것이 쉽게 합니다. Athena는 serverless이므로 관리를 위한 infrastructure가 필요없으며, queries한 부분에 대해 비용이 과금됩니다.

Athena는 Presto 기반으로, CSV, JSON, Avro 외에도 Apache Parquet와 Apache ORC 와 같은 columnar data도 지원합니다.

Presto는 모든 크기의 데이터에 대해 빠른 분석 쿼리를 위해 개발 된 오픈 소스 분산 SQL 쿼리 엔진입니다. 데이터를 이동할 필요없이 저장된 위치의 데이터를 쿼리 할 수 있습니다. 쿼리 실행은 순수한 메모리 기반 아키텍처에서 병렬로 실행되므로 Presto가 매우 빠릅니다.


In [ ]:
%store -r

In [ ]:
import sys
import boto3
import sagemaker

<p>SageMaker에서 앞으로 사용할 SageMaker Session 설정, Role 정보를 설정합니다. </p>

In [ ]:
sagemaker_session = sagemaker.Session()
sess = boto3.Session()
sm = sess.client('sagemaker')

## 1 ) Install PyAthena

[PyAthena](https://pypi.org/project/PyAthena/) 는 Amazon Athena용 Python DB API 2.0 (PEP 249) 호환 클라이언트 입니다.

In [ ]:
# Install PyAthena
!{sys.executable} -m pip install -q --upgrade pip
!{sys.executable} -m pip install -q PyAthena==1.10.7

In [ ]:
from pyathena import connect
from pyathena.pandas_cursor import PandasCursor
from pyathena.util import as_pandas

## 2 ) Create Athena Database

In [ ]:
# Set Athena database name
database_name = 'awsdb'

Athena에서 생성 한 데이터베이스 및 테이블은 Data Catalog 서비스를 사용하여 데이터의 메타 데이터를 저장합니다. 예를 들어, 테이블 이름과 함께 테이블에 있는 각 열의 데이터 이름과 열 이름으로 구성된 스키마 정보는 Data Catalog에 메타 데이터 정보로 저장됩니다.

Athena는 기본적으로 AWS Glue Data Catalog 서비스를 지원합니다. AWS Glue Data Catalog를 소스로 사용하여 Athena에서 `CREATE DATABASE` 및 `CREATE TABLE` 쿼리를 실행하면 AWS Glue Data Catalog에서 생성되는 데이터베이스 및 테이블 메타 데이터 항목이 자동으로 표시됩니다.

In [ ]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = 's3://{0}/athena/staging'.format(job_bucket)

In [ ]:
# SQL statement to execute
statement = 'DROP SCHEMA IF EXISTS {} CASCADE'.format(database_name)
print(statement)

# Execute statement using connection cursor
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

In [ ]:
# SQL statement to execute
statement = 'CREATE DATABASE IF NOT EXISTS {}'.format(database_name)
print(statement)

In [ ]:
# Execute statement using connection cursor
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

#### 데이터베이스가 성공적으로 작성되었는지 확인합니다.

In [ ]:
statement = 'SHOW DATABASES'
cursor.execute(statement)

df_show = as_pandas(cursor)
df_show.head(5)

# 2. Amazon Athena에 Data 등록

In [ ]:
table_name_tsv = 'amazon_reviews_tsv'

In [ ]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         marketplace string,
         customer_id string,
         review_id string,
         product_id string,
         product_parent string,
         product_title string,
         product_category string,
         star_rating int,
         helpful_votes int,
         total_votes int,
         vine string,
         verified_purchase string,
         review_headline string,
         review_body string,
         review_date string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\\t' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(database_name, table_name_tsv, s3_destination_path_tsv)

print(statement)

In [ ]:
# Execute statement using connection cursor
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

#### 테이블 생성을 확인합니다.

In [ ]:
statement = 'SHOW TABLES in {}'.format(database_name)
cursor.execute(statement)

df_show = as_pandas(cursor)
df_show.head(5)

#### Sample query를 수행합니다.

In [ ]:
product_category = 'Digital_Software'

statement = """SELECT * FROM {}.{}
    WHERE product_category = '{}' LIMIT 100""".format(database_name, table_name_tsv, product_category)

print(statement)

In [ ]:
# Execute statement using connection cursor
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

In [ ]:
df = as_pandas(cursor)
df.head(5)

In [ ]:
%store s3_staging_dir database_name table_name_tsv